In [1]:
import pandas as pd
import os
import time
import fnmatch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
fixlen_path = '/home/jxin05/BBMAS_Data/fixlen_data/'

def remove_col(df):
    return df.drop(['EID','time','time_in_ms'], axis =1)

# this function split pure data samples into 70%/10%/20% as training, validation 
# and testing sets
def train_test_split(path):
    file_list = os.listdir(path)
    num_sample = len(file_list)
    idx = np.random.permutation(num_sample)
    train_idx = idx[:int(num_sample*0.7)]
    validate_idx = idx[int(num_sample*0.7) : int(num_sample*0.8)]
    test_idx = idx[int(num_sample*0.8):]
    trainfile = []
    validatefile = []
    testfile = []
    for i in train_idx:
        trainfile.append(file_list[i])
    for i in validate_idx:
        validatefile.append(file_list[i])
    for i in test_idx:
        testfile.append(file_list[i])
    return trainfile, validatefile, testfile

# returns a list of synthetic data with size 3x12000
# pick_dict is like {0: [10,21,13,44...], 1: [5,7,9,10...] } where each element 
# of the list (value of a key) is the index(from 3 to 58) of replaced part
def mix_data(path, filelist, fake_number, replace_time, replace_size = 200):
    pick_dict = {}
    mixed_data = []
    count = 0
    while(count < fake_number):
        pick = np.random.randint(0, len(filelist))
        pick_dict[count] = []
        target = pd.read_csv(path + filelist[pick])
        mix = np.asarray(remove_col(target).values)
        for j in range(replace_time):
            start = np.random.randint(3,len(target)/replace_size-1)
            while(start in pick_dict[count]):
                start = np.random.randint(3,len(target)/replace_size-1)
            pick_dict[count].append(start)
            replace_clip = take_from_others(path, filelist,replace_size, pick)
            mix[start*replace_size:(start+1)*replace_size, :] = replace_clip
        mixed_data.append(mix)
        count+=1
    return mixed_data, pick_dict 
 
# randomly take a clip from a sample
# pick is the index of data sample to be processed
def take_from_others(path, filelist, replace_size, pick):
    takefrom = np.random.randint(0,len(filelist))
    while(takefrom == pick):
        takefrom = np.random.randint(0,len(filelist))
    target_df = pd.read_csv(path+filelist[takefrom])
    start_idx = np.random.randint(0,len(target_df)-replace_size)
    replace_clip = target_df.iloc[start_idx:start_idx+replace_size]
    replace_clip = remove_col(replace_clip)
    return np.asarray(replace_clip.values)

# load pure data and make its label vector
def load_original_data(path,filelist, fix_len):
    X = np.zeros((len(filelist), 3, fix_len))
    Y = np.zeros((len(filelist), 60))    
    for i in range(len(filelist)):
        x = remove_col(pd.read_csv(path+filelist[i])).values
        X[i,:,:] = x.T
    return X,Y

# smooth boundary between pure signal and synthetic signal
# apply ExpMovingAverage on 20 data points with window 3
def smooth(mixed_data, pick_dict, window = 3):
    res = mixed_data
    for i in range(len(mixed_data)):
        for v in pick_dict[i]:
            #print(v)
            start = v*200-10
            end = v*200+10
            data_x = mixed_data[i][start:end,0]
            #print(data_x)
            data_y = mixed_data[i][start:end,1]
            data_z = mixed_data[i][start:end,2]
            smoothed_x = ExpMovingAverage(data_x, window)
            #print(smoothed_x)
            smoothed_y = ExpMovingAverage(data_y, window)
            smoothed_z = ExpMovingAverage(data_z, window)
            res[i][start+window+1:end-window,0] = smoothed_x[window+1:-window]
            #print(res[i][start+window+1:end-window,0])
            res[i][start+window+1:end-window,1] = smoothed_y[window+1:-window]
            res[i][start+window+1:end-window,2] = smoothed_z[window+1:-window]
    return res
def ExpMovingAverage(array, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a = np.convolve(array, weights, mode='full')[:len(array)]
    a[:window] = a[window]
    return a

def make_fake_dataset(mixed_data, pick_dict):
    sample_len = mixed_data[0].shape[0]
    num_sample = len(mixed_data)
    X = np.zeros((num_sample, mixed_data[0].shape[1], sample_len))
    Y = np.zeros((num_sample, 60))
    for i in range(len(mixed_data)):
        X[i,:,:] = mixed_data[i].T
        for v in pick_dict[i]:
            Y[i,v] = 1
    return X,Y

# reshape input data to fit network
def transformation(x):
    if(len(x.shape) == 3):
        inputslist = []
        for i in range(x.shape[0]):
            inputs_x = torch.from_numpy(x[i,0,:]).float()
            inputs_x = inputs_x.view(60,-1)
            inputs_y = torch.from_numpy(x[i,1,:]).float()
            inputs_y = inputs_y.view(60,-1)
            inputs_z = torch.from_numpy(x[i,2,:]).float()
            inputs_z = inputs_z.view(60,-1)
            inputs = torch.stack([inputs_x, inputs_y, inputs_z])
            inputs = inputs.transpose(0,1)
            inputslist.append(inputs)
        res = torch.stack(inputslist)
        res = res.view(x.shape[0], 60, 1, 3, 200)
        return res
    else:
        inputs_x = torch.from_numpy(x[0,:]).float()
        inputs_x = inputs_x.view(60,-1)
        inputs_y = torch.from_numpy(x[1,:]).float()
        inputs_y = inputs_y.view(60,-1)
        inputs_z = torch.from_numpy(x[2,:]).float()
        inputs_z = inputs_z.view(60,-1)
        inputs = torch.stack([inputs_x, inputs_y, inputs_z])
        inputs = inputs.transpose(0,1)
        inputs = inputs.view(-1, 60, 1, 3, 200)
        return inputs
# generate a batch 
def take_batch(batch_size, X, Y):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for i in range(0,X.shape[0]-batch_size+1, batch_size):
        excerpt = indices[i:i + batch_size]
        yield X[excerpt], Y[excerpt]

In [2]:
def compute_accuracy(net, x, y, batch_size = 1):
    with torch.no_grad():
        h = net.init_hidden(batch_size)
        output, h = net(x, h)
        output = output.squeeze()
    label = []
    for i in range(len(output)):
        if(output[i,0] >= output[i,1]):
            label.append(0)
        else:
            label.append(1)
    label = np.asarray(label)
    acc = len(y[label == y])/len(y)
    return acc

def check_performance(net, X, Y):
    acc_list = []
    for i in range(X.shape[0]):
        acc = compute_accuracy(net, X[i], Y[i])
        acc_list.append(acc)
    return np.mean(acc_list)

def train_batch_m2m(net, trainX,trainY, valX, valY, 
                    batch_size = 20, epochs = 250, lr = 0.006):
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    val_acc = 0
    best_model = None
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        train_loss = []
        net.train()
        for x,y in take_batch(batch_size, trainX, trainY):
            targets = torch.from_numpy(y).view(batch_size, -1)
            #print(targets.size())
            h = tuple([each.data for each in h])
            opt.zero_grad()  
            output, h = net(x, h)
            #print(output.size())
            
            loss = criterion(output, targets.long())
            train_loss.append(loss.item())
            loss.backward()
            opt.step()
        acc = check_performance(net, valX, valY)
        if(acc >= val_acc): # store the model performs best on validation set
            best_model = net
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_loss)),
              "Validation acc: {:.4f}...".format(acc))
    final_model = net
    return best_model, final_model

In [3]:
def onerun(net, path):
        print("creating synthetic data")
        trainfile, validatefile, testfile = train_test_split(path)
        train_data1, train_dict1 = mix_data(path, trainfile, 210, 30) 
        train_data2, train_dict2 = mix_data(path, trainfile, 210, 20)
        train_data3, train_dict3 = mix_data(path, trainfile, 210, 15) 
        train_data4, train_dict4 = mix_data(path, trainfile, 210, 10)
        validate_data1, validate_dict1 = mix_data(path, validatefile, 30, 30)
        validate_data2, validate_dict2 = mix_data(path, validatefile, 30, 20)
        validate_data3, validate_dict3 = mix_data(path, validatefile, 30, 15)
        validate_data4, validate_dict4 = mix_data(path, validatefile, 30, 10) 
        test_data1, test_dict1 = mix_data(path, testfile, 60, 30) 
        test_data2, test_dict2 = mix_data(path, testfile, 60, 20)
        test_data3, test_dict3 = mix_data(path, testfile, 60, 15) 
        test_data4, test_dict4 = mix_data(path, testfile, 60, 10) 
        print("smoothing data")
        train_data1 = smooth(train_data1, train_dict1)
        train_data2 = smooth(train_data2, train_dict2)
        train_data3 = smooth(train_data3, train_dict3)
        train_data4 = smooth(train_data4, train_dict4)
        validate_data1 = smooth(validate_data1, validate_dict1)
        validate_data2 = smooth(validate_data2, validate_dict2)
        validate_data3 = smooth(validate_data3, validate_dict3)
        validate_data4 = smooth(validate_data4, validate_dict4)
        test_data1 = smooth(test_data1, test_dict1)
        test_data2 = smooth(test_data2, test_dict2)
        test_data3 = smooth(test_data3, test_dict3)
        test_data4 = smooth(test_data4, test_dict4)
        print("making datasets")
        original_trainX, original_trainY = load_original_data(path,trainfile,12000)
        original_testX, original_testY = load_original_data(path,testfile,12000)
        original_valX, original_valY = load_original_data(path,validatefile, 12000)
        
        X1_train, Y1_train = make_fake_dataset(train_data1, train_dict1)
        X2_train, Y2_train = make_fake_dataset(train_data2, train_dict2)
        X3_train, Y3_train = make_fake_dataset(train_data3, train_dict3)
        X4_train, Y4_train = make_fake_dataset(train_data4, train_dict4)
        
        X1_val, Y1_val = make_fake_dataset(validate_data1, validate_dict1)
        X2_val, Y2_val = make_fake_dataset(validate_data2, validate_dict2)
        X3_val, Y3_val = make_fake_dataset(validate_data3, validate_dict3)
        X4_val, Y4_val = make_fake_dataset(validate_data4, validate_dict4)
        
        X1_test, Y1_test = make_fake_dataset(test_data1, test_dict1)
        X2_test, Y2_test = make_fake_dataset(test_data2, test_dict2)
        X3_test, Y3_test = make_fake_dataset(test_data3, test_dict3)
        X4_test, Y4_test = make_fake_dataset(test_data4, test_dict4)
        train_X = np.vstack([X1_train, X2_train, X3_train, original_trainX, X4_train])
        train_Y = np.vstack([Y1_train, Y2_train, Y3_train, original_trainY, Y4_train])
        test_X = np.vstack([X1_test, X2_test, X3_test, original_testX, X4_test])
        test_Y = np.vstack([Y1_test, Y2_test, Y3_test, original_testY, Y4_test])
        val_X = np.vstack([X1_val, X2_val, X3_val, original_valX, X4_val])
        val_Y = np.vstack([Y1_val, Y2_val, Y3_val, original_valY, Y4_val])
        #data preparing complete!
        best, final = train_batch_m2m(net, train_X, train_Y, val_X, val_Y)
        acc1 = check_performance(final, X1_test, Y1_test)
        acc2 = check_performance(final, X2_test, Y2_test)
        acc3 = check_performance(final, X3_test, Y3_test)
        acc4 = check_performance(final, X4_test, Y4_test)
        acc5 = check_performance(final, original_testX, original_testY)
        res_final = [acc1, acc2, acc3, acc4, acc5]
        
        acc1 = check_performance(best, X1_test, Y1_test)
        acc2 = check_performance(best, X2_test, Y2_test)
        acc3 = check_performance(best, X3_test, Y3_test)
        acc4 = check_performance(best, X4_test, Y4_test)
        acc5 = check_performance(best, original_testX, original_testY)
        res_best = [acc1, acc2, acc3, acc4, acc5]
        return res_best, res_final, best, final

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,32,(2,8), stride = (1,4))
        self.conv2 = nn.Conv2d(32,64, (2,4), stride = (1,2))
        self.conv3 = nn.Conv1d(64,32, 4, stride = 2)
    
    def forward(self, x):
        inputs = self.conv1(x)
        inputs = F.relu(inputs)
        inputs = self.conv2(inputs)
        inputs = F.relu(inputs)
        inputs = inputs.squeeze()
        inputs = self.conv3(inputs)
        inputs = F.relu(inputs)
        
        return inputs

class cnn_lstm(nn.Module):
    def __init__(self, cnn = None, drop_prob = 0.3, n_class = 2, n_layer = 1):
        super(cnn_lstm, self).__init__()
        if(cnn == None):
            self.cnn = CNN()
        else:
            self.cnn = cnn
        self.n_class = n_class
        self.lstm = nn.LSTM(320, 128, num_layers = n_layer, batch_first = True)
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32,n_class)
        self.dropout = nn.Dropout(drop_prob)
        self.outlayer = nn.Softmax(dim = 2)
    
    def forward(self, x, hidden):
        inputs = transformation(x)
        batch_size, timesteps, C, H, W = inputs.size()
        inputs = inputs.view(batch_size*timesteps,C,H,W)
        c_out = self.cnn(inputs)
        lstm_in = c_out.view(batch_size, timesteps, -1)
        lstm_out, hidden = self.lstm(lstm_in,hidden)
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.outlayer(out)
        
        if(batch_size != 1):
            out = out.transpose(1,2)
        return out, hidden
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(1, batch_size, 128).zero_(),
                      weight.new(1, batch_size, 128).zero_())
        return hidden

def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv2d or type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
        
def init_weights2(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)

In [5]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6006... Validation acc: 0.7221...
Epoch: 2/250... Train Loss: 0.5909... Validation acc: 0.7222...
Epoch: 3/250... Train Loss: 0.5907... Validation acc: 0.7221...
Epoch: 4/250... Train Loss: 0.5905... Validation acc: 0.7222...
Epoch: 5/250... Train Loss: 0.5892... Validation acc: 0.7223...
Epoch: 6/250... Train Loss: 0.5842... Validation acc: 0.7283...
Epoch: 7/250... Train Loss: 0.5654... Validation acc: 0.7932...
Epoch: 8/250... Train Loss: 0.5466... Validation acc: 0.7984...
Epoch: 9/250... Train Loss: 0.5232... Validation acc: 0.8593...
Epoch: 10/250... Train Loss: 0.5053... Validation acc: 0.8632...
Epoch: 11/250... Train Loss: 0.4991... Validation acc: 0.8612...
Epoch: 12/250... Train Loss: 0.4944... Validation acc: 0.8643...
Epoch: 13/250... Train Loss: 0.4927... Validation acc: 0.8656...
Epoch: 14/250... Train Loss: 0.4932... Validation acc: 0.8633...
Epoch: 15/250... Train Loss: 0.4921... Valid

Epoch: 126/250... Train Loss: 0.4309... Validation acc: 0.8910...
Epoch: 127/250... Train Loss: 0.4303... Validation acc: 0.8909...
Epoch: 128/250... Train Loss: 0.4308... Validation acc: 0.8922...
Epoch: 129/250... Train Loss: 0.4306... Validation acc: 0.8888...
Epoch: 130/250... Train Loss: 0.4305... Validation acc: 0.8917...
Epoch: 131/250... Train Loss: 0.4306... Validation acc: 0.8904...
Epoch: 132/250... Train Loss: 0.4306... Validation acc: 0.8915...
Epoch: 133/250... Train Loss: 0.4298... Validation acc: 0.8907...
Epoch: 134/250... Train Loss: 0.4304... Validation acc: 0.8910...
Epoch: 135/250... Train Loss: 0.4307... Validation acc: 0.8895...
Epoch: 136/250... Train Loss: 0.4305... Validation acc: 0.8917...
Epoch: 137/250... Train Loss: 0.4300... Validation acc: 0.8911...
Epoch: 138/250... Train Loss: 0.4297... Validation acc: 0.8868...
Epoch: 139/250... Train Loss: 0.4301... Validation acc: 0.8902...
Epoch: 140/250... Train Loss: 0.4294... Validation acc: 0.8900...
Epoch: 141

[0.8305555555555556, 0.8847222222222223, 0.8877777777777778, 0.9202777777777779, 0.9738888888888888]
[0.8302777777777777, 0.8861111111111111, 0.8897222222222222, 0.9216666666666667, 0.9755555555555555]


In [6]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.5979... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5836... Validation acc: 0.7315...
Epoch: 3/250... Train Loss: 0.5577... Validation acc: 0.7893...
Epoch: 4/250... Train Loss: 0.5190... Validation acc: 0.8540...
Epoch: 5/250... Train Loss: 0.5003... Validation acc: 0.8560...
Epoch: 6/250... Train Loss: 0.4942... Validation acc: 0.8556...
Epoch: 7/250... Train Loss: 0.4932... Validation acc: 0.8570...
Epoch: 8/250... Train Loss: 0.4910... Validation acc: 0.8567...
Epoch: 9/250... Train Loss: 0.4901... Validation acc: 0.8575...
Epoch: 10/250... Train Loss: 0.4894... Validation acc: 0.8605...
Epoch: 11/250... Train Loss: 0.4902... Validation acc: 0.8552...
Epoch: 12/250... Train Loss: 0.4902... Validation acc: 0.8567...
Epoch: 13/250... Train Loss: 0.4874... Validation acc: 0.8579...
Epoch: 14/250... Train Loss: 0.4880... Validation acc: 0.8564...
Epoch: 15/250... Train Loss: 0.4884... Valid

Epoch: 126/250... Train Loss: 0.4201... Validation acc: 0.8891...
Epoch: 127/250... Train Loss: 0.4190... Validation acc: 0.8916...
Epoch: 128/250... Train Loss: 0.4186... Validation acc: 0.8920...
Epoch: 129/250... Train Loss: 0.4165... Validation acc: 0.8912...
Epoch: 130/250... Train Loss: 0.4159... Validation acc: 0.8938...
Epoch: 131/250... Train Loss: 0.4152... Validation acc: 0.8948...
Epoch: 132/250... Train Loss: 0.4141... Validation acc: 0.8960...
Epoch: 133/250... Train Loss: 0.4119... Validation acc: 0.8978...
Epoch: 134/250... Train Loss: 0.4109... Validation acc: 0.8993...
Epoch: 135/250... Train Loss: 0.4094... Validation acc: 0.9000...
Epoch: 136/250... Train Loss: 0.4093... Validation acc: 0.8979...
Epoch: 137/250... Train Loss: 0.4089... Validation acc: 0.8979...
Epoch: 138/250... Train Loss: 0.4063... Validation acc: 0.9007...
Epoch: 139/250... Train Loss: 0.4061... Validation acc: 0.8969...
Epoch: 140/250... Train Loss: 0.4051... Validation acc: 0.9011...
Epoch: 141

[0.9205555555555556, 0.9355555555555557, 0.9369444444444444, 0.9397222222222222, 0.9538888888888889]
[0.9197222222222222, 0.9358333333333333, 0.9369444444444446, 0.9375, 0.9538888888888889]


In [7]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6032... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5827... Validation acc: 0.7378...
Epoch: 3/250... Train Loss: 0.5549... Validation acc: 0.7677...
Epoch: 4/250... Train Loss: 0.5191... Validation acc: 0.8451...
Epoch: 5/250... Train Loss: 0.4989... Validation acc: 0.8581...
Epoch: 6/250... Train Loss: 0.4965... Validation acc: 0.8609...
Epoch: 7/250... Train Loss: 0.4916... Validation acc: 0.8600...
Epoch: 8/250... Train Loss: 0.4909... Validation acc: 0.8594...
Epoch: 9/250... Train Loss: 0.4913... Validation acc: 0.8606...
Epoch: 10/250... Train Loss: 0.4890... Validation acc: 0.8572...
Epoch: 11/250... Train Loss: 0.4865... Validation acc: 0.8630...
Epoch: 12/250... Train Loss: 0.4883... Validation acc: 0.8589...
Epoch: 13/250... Train Loss: 0.4881... Validation acc: 0.8594...
Epoch: 14/250... Train Loss: 0.4884... Validation acc: 0.8605...
Epoch: 15/250... Train Loss: 0.4867... Valid

Epoch: 126/250... Train Loss: 0.4277... Validation acc: 0.8991...
Epoch: 127/250... Train Loss: 0.4275... Validation acc: 0.8985...
Epoch: 128/250... Train Loss: 0.4274... Validation acc: 0.8990...
Epoch: 129/250... Train Loss: 0.4272... Validation acc: 0.8996...
Epoch: 130/250... Train Loss: 0.4270... Validation acc: 0.8993...
Epoch: 131/250... Train Loss: 0.4265... Validation acc: 0.9000...
Epoch: 132/250... Train Loss: 0.4258... Validation acc: 0.8979...
Epoch: 133/250... Train Loss: 0.4267... Validation acc: 0.8986...
Epoch: 134/250... Train Loss: 0.4259... Validation acc: 0.8985...
Epoch: 135/250... Train Loss: 0.4256... Validation acc: 0.9005...
Epoch: 136/250... Train Loss: 0.4247... Validation acc: 0.8991...
Epoch: 137/250... Train Loss: 0.4250... Validation acc: 0.9006...
Epoch: 138/250... Train Loss: 0.4241... Validation acc: 0.9014...
Epoch: 139/250... Train Loss: 0.4239... Validation acc: 0.9007...
Epoch: 140/250... Train Loss: 0.4227... Validation acc: 0.8965...
Epoch: 141

[0.8883333333333334, 0.8808333333333332, 0.8958333333333334, 0.9180555555555557, 0.957777777777778]
[0.8866666666666667, 0.8833333333333334, 0.8952777777777777, 0.9208333333333333, 0.9600000000000001]


In [8]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6016... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5901... Validation acc: 0.7222...
Epoch: 3/250... Train Loss: 0.5883... Validation acc: 0.7222...
Epoch: 4/250... Train Loss: 0.5836... Validation acc: 0.7254...
Epoch: 5/250... Train Loss: 0.5651... Validation acc: 0.7656...
Epoch: 6/250... Train Loss: 0.5460... Validation acc: 0.7757...
Epoch: 7/250... Train Loss: 0.5227... Validation acc: 0.8170...
Epoch: 8/250... Train Loss: 0.5047... Validation acc: 0.8258...
Epoch: 9/250... Train Loss: 0.4977... Validation acc: 0.8247...
Epoch: 10/250... Train Loss: 0.4959... Validation acc: 0.8260...
Epoch: 11/250... Train Loss: 0.4947... Validation acc: 0.8246...
Epoch: 12/250... Train Loss: 0.4927... Validation acc: 0.8235...
Epoch: 13/250... Train Loss: 0.4930... Validation acc: 0.8220...
Epoch: 14/250... Train Loss: 0.4915... Validation acc: 0.8248...
Epoch: 15/250... Train Loss: 0.4909... Valid

Epoch: 126/250... Train Loss: 0.4249... Validation acc: 0.8740...
Epoch: 127/250... Train Loss: 0.4244... Validation acc: 0.8701...
Epoch: 128/250... Train Loss: 0.4243... Validation acc: 0.8704...
Epoch: 129/250... Train Loss: 0.4227... Validation acc: 0.8737...
Epoch: 130/250... Train Loss: 0.4221... Validation acc: 0.8728...
Epoch: 131/250... Train Loss: 0.4225... Validation acc: 0.8741...
Epoch: 132/250... Train Loss: 0.4207... Validation acc: 0.8747...
Epoch: 133/250... Train Loss: 0.4218... Validation acc: 0.8705...
Epoch: 134/250... Train Loss: 0.4218... Validation acc: 0.8733...
Epoch: 135/250... Train Loss: 0.4206... Validation acc: 0.8731...
Epoch: 136/250... Train Loss: 0.4210... Validation acc: 0.8714...
Epoch: 137/250... Train Loss: 0.4205... Validation acc: 0.8754...
Epoch: 138/250... Train Loss: 0.4190... Validation acc: 0.8719...
Epoch: 139/250... Train Loss: 0.4189... Validation acc: 0.8758...
Epoch: 140/250... Train Loss: 0.4187... Validation acc: 0.8721...
Epoch: 141

[0.8549999999999999, 0.8761111111111111, 0.8991666666666664, 0.9161111111111111, 0.9755555555555555]
[0.8574999999999998, 0.8780555555555556, 0.8980555555555554, 0.9158333333333333, 0.9744444444444444]


In [9]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6054... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5848... Validation acc: 0.7228...
Epoch: 3/250... Train Loss: 0.5535... Validation acc: 0.8340...
Epoch: 4/250... Train Loss: 0.5088... Validation acc: 0.8598...
Epoch: 5/250... Train Loss: 0.4968... Validation acc: 0.8588...
Epoch: 6/250... Train Loss: 0.4941... Validation acc: 0.8640...
Epoch: 7/250... Train Loss: 0.4910... Validation acc: 0.8612...
Epoch: 8/250... Train Loss: 0.4906... Validation acc: 0.8573...
Epoch: 9/250... Train Loss: 0.4891... Validation acc: 0.8577...
Epoch: 10/250... Train Loss: 0.4873... Validation acc: 0.8627...
Epoch: 11/250... Train Loss: 0.4891... Validation acc: 0.8602...
Epoch: 12/250... Train Loss: 0.4880... Validation acc: 0.8589...
Epoch: 13/250... Train Loss: 0.4898... Validation acc: 0.8598...
Epoch: 14/250... Train Loss: 0.4867... Validation acc: 0.8617...
Epoch: 15/250... Train Loss: 0.4875... Valid

Epoch: 126/250... Train Loss: 0.4085... Validation acc: 0.9014...
Epoch: 127/250... Train Loss: 0.4099... Validation acc: 0.9022...
Epoch: 128/250... Train Loss: 0.4084... Validation acc: 0.9033...
Epoch: 129/250... Train Loss: 0.4071... Validation acc: 0.9053...
Epoch: 130/250... Train Loss: 0.4070... Validation acc: 0.9047...
Epoch: 131/250... Train Loss: 0.4072... Validation acc: 0.9021...
Epoch: 132/250... Train Loss: 0.4065... Validation acc: 0.9065...
Epoch: 133/250... Train Loss: 0.4057... Validation acc: 0.9044...
Epoch: 134/250... Train Loss: 0.4057... Validation acc: 0.9057...
Epoch: 135/250... Train Loss: 0.4053... Validation acc: 0.9070...
Epoch: 136/250... Train Loss: 0.4044... Validation acc: 0.9086...
Epoch: 137/250... Train Loss: 0.4046... Validation acc: 0.9058...
Epoch: 138/250... Train Loss: 0.4045... Validation acc: 0.9070...
Epoch: 139/250... Train Loss: 0.4028... Validation acc: 0.9078...
Epoch: 140/250... Train Loss: 0.4029... Validation acc: 0.9083...
Epoch: 141

[0.9172222222222222, 0.9277777777777778, 0.9280555555555555, 0.9413888888888889, 0.9594444444444442]
[0.9186111111111114, 0.9305555555555556, 0.9272222222222222, 0.9386111111111112, 0.9594444444444444]


In [10]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.5995... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5787... Validation acc: 0.7420...
Epoch: 3/250... Train Loss: 0.5441... Validation acc: 0.8148...
Epoch: 4/250... Train Loss: 0.5144... Validation acc: 0.8372...
Epoch: 5/250... Train Loss: 0.5022... Validation acc: 0.8407...
Epoch: 6/250... Train Loss: 0.4944... Validation acc: 0.8422...
Epoch: 7/250... Train Loss: 0.4943... Validation acc: 0.8474...
Epoch: 8/250... Train Loss: 0.4927... Validation acc: 0.8488...
Epoch: 9/250... Train Loss: 0.4952... Validation acc: 0.8494...
Epoch: 10/250... Train Loss: 0.4917... Validation acc: 0.8509...
Epoch: 11/250... Train Loss: 0.4948... Validation acc: 0.8483...
Epoch: 12/250... Train Loss: 0.4905... Validation acc: 0.8454...
Epoch: 13/250... Train Loss: 0.4923... Validation acc: 0.8512...
Epoch: 14/250... Train Loss: 0.4920... Validation acc: 0.8547...
Epoch: 15/250... Train Loss: 0.4896... Valid

Epoch: 126/250... Train Loss: 0.4233... Validation acc: 0.8928...
Epoch: 127/250... Train Loss: 0.4231... Validation acc: 0.8946...
Epoch: 128/250... Train Loss: 0.4226... Validation acc: 0.8890...
Epoch: 129/250... Train Loss: 0.4222... Validation acc: 0.8880...
Epoch: 130/250... Train Loss: 0.4213... Validation acc: 0.8900...
Epoch: 131/250... Train Loss: 0.4219... Validation acc: 0.8927...
Epoch: 132/250... Train Loss: 0.4220... Validation acc: 0.8930...
Epoch: 133/250... Train Loss: 0.4218... Validation acc: 0.8923...
Epoch: 134/250... Train Loss: 0.4214... Validation acc: 0.8968...
Epoch: 135/250... Train Loss: 0.4209... Validation acc: 0.8922...
Epoch: 136/250... Train Loss: 0.4208... Validation acc: 0.8970...
Epoch: 137/250... Train Loss: 0.4210... Validation acc: 0.8927...
Epoch: 138/250... Train Loss: 0.4208... Validation acc: 0.8911...
Epoch: 139/250... Train Loss: 0.4203... Validation acc: 0.8940...
Epoch: 140/250... Train Loss: 0.4192... Validation acc: 0.8970...
Epoch: 141

[0.8872222222222224, 0.9019444444444445, 0.9083333333333334, 0.9372222222222224, 0.9827777777777779]
[0.8902777777777778, 0.9041666666666667, 0.9138888888888891, 0.9375000000000001, 0.9822222222222222]


In [11]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.5999... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5904... Validation acc: 0.7222...
Epoch: 3/250... Train Loss: 0.5849... Validation acc: 0.7231...
Epoch: 4/250... Train Loss: 0.5552... Validation acc: 0.8119...
Epoch: 5/250... Train Loss: 0.5148... Validation acc: 0.8586...
Epoch: 6/250... Train Loss: 0.5026... Validation acc: 0.8600...
Epoch: 7/250... Train Loss: 0.4985... Validation acc: 0.8594...
Epoch: 8/250... Train Loss: 0.4957... Validation acc: 0.8595...
Epoch: 9/250... Train Loss: 0.4920... Validation acc: 0.8614...
Epoch: 10/250... Train Loss: 0.4919... Validation acc: 0.8586...
Epoch: 11/250... Train Loss: 0.4920... Validation acc: 0.8564...
Epoch: 12/250... Train Loss: 0.4898... Validation acc: 0.8577...
Epoch: 13/250... Train Loss: 0.4887... Validation acc: 0.8599...
Epoch: 14/250... Train Loss: 0.4881... Validation acc: 0.8583...
Epoch: 15/250... Train Loss: 0.4892... Valid

Epoch: 126/250... Train Loss: 0.4220... Validation acc: 0.8906...
Epoch: 127/250... Train Loss: 0.4232... Validation acc: 0.8900...
Epoch: 128/250... Train Loss: 0.4223... Validation acc: 0.8886...
Epoch: 129/250... Train Loss: 0.4212... Validation acc: 0.8905...
Epoch: 130/250... Train Loss: 0.4212... Validation acc: 0.8930...
Epoch: 131/250... Train Loss: 0.4211... Validation acc: 0.8919...
Epoch: 132/250... Train Loss: 0.4202... Validation acc: 0.8927...
Epoch: 133/250... Train Loss: 0.4196... Validation acc: 0.8948...
Epoch: 134/250... Train Loss: 0.4190... Validation acc: 0.8933...
Epoch: 135/250... Train Loss: 0.4189... Validation acc: 0.8926...
Epoch: 136/250... Train Loss: 0.4179... Validation acc: 0.8963...
Epoch: 137/250... Train Loss: 0.4176... Validation acc: 0.8942...
Epoch: 138/250... Train Loss: 0.4182... Validation acc: 0.8967...
Epoch: 139/250... Train Loss: 0.4166... Validation acc: 0.8957...
Epoch: 140/250... Train Loss: 0.4164... Validation acc: 0.8978...
Epoch: 141

[0.8833333333333332, 0.8880555555555556, 0.9105555555555556, 0.9344444444444445, 0.976111111111111]
[0.8833333333333333, 0.8866666666666667, 0.91, 0.9372222222222221, 0.9772222222222222]


In [12]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6053... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5882... Validation acc: 0.7223...
Epoch: 3/250... Train Loss: 0.5792... Validation acc: 0.7453...
Epoch: 4/250... Train Loss: 0.5472... Validation acc: 0.8044...
Epoch: 5/250... Train Loss: 0.5149... Validation acc: 0.8207...
Epoch: 6/250... Train Loss: 0.5010... Validation acc: 0.8237...
Epoch: 7/250... Train Loss: 0.4964... Validation acc: 0.8258...
Epoch: 8/250... Train Loss: 0.4949... Validation acc: 0.8277...
Epoch: 9/250... Train Loss: 0.4957... Validation acc: 0.8243...
Epoch: 10/250... Train Loss: 0.4944... Validation acc: 0.8237...
Epoch: 11/250... Train Loss: 0.4919... Validation acc: 0.8263...
Epoch: 12/250... Train Loss: 0.4925... Validation acc: 0.8260...
Epoch: 13/250... Train Loss: 0.4921... Validation acc: 0.8240...
Epoch: 14/250... Train Loss: 0.4918... Validation acc: 0.8254...
Epoch: 15/250... Train Loss: 0.4909... Valid

Epoch: 126/250... Train Loss: 0.4292... Validation acc: 0.8474...
Epoch: 127/250... Train Loss: 0.4293... Validation acc: 0.8470...
Epoch: 128/250... Train Loss: 0.4294... Validation acc: 0.8478...
Epoch: 129/250... Train Loss: 0.4281... Validation acc: 0.8469...
Epoch: 130/250... Train Loss: 0.4274... Validation acc: 0.8474...
Epoch: 131/250... Train Loss: 0.4280... Validation acc: 0.8477...
Epoch: 132/250... Train Loss: 0.4281... Validation acc: 0.8474...
Epoch: 133/250... Train Loss: 0.4276... Validation acc: 0.8477...
Epoch: 134/250... Train Loss: 0.4269... Validation acc: 0.8467...
Epoch: 135/250... Train Loss: 0.4272... Validation acc: 0.8480...
Epoch: 136/250... Train Loss: 0.4265... Validation acc: 0.8469...
Epoch: 137/250... Train Loss: 0.4274... Validation acc: 0.8474...
Epoch: 138/250... Train Loss: 0.4265... Validation acc: 0.8477...
Epoch: 139/250... Train Loss: 0.4259... Validation acc: 0.8483...
Epoch: 140/250... Train Loss: 0.4256... Validation acc: 0.8478...
Epoch: 141

[0.8647222222222223, 0.8905555555555554, 0.9041666666666667, 0.9269444444444446, 0.9794444444444445]
[0.8661111111111112, 0.8947222222222222, 0.906111111111111, 0.9286111111111113, 0.9833333333333335]


In [13]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6079... Validation acc: 0.7225...
Epoch: 2/250... Train Loss: 0.5636... Validation acc: 0.7796...
Epoch: 3/250... Train Loss: 0.5270... Validation acc: 0.8285...
Epoch: 4/250... Train Loss: 0.5040... Validation acc: 0.8309...
Epoch: 5/250... Train Loss: 0.4946... Validation acc: 0.8264...
Epoch: 6/250... Train Loss: 0.4921... Validation acc: 0.8289...
Epoch: 7/250... Train Loss: 0.4918... Validation acc: 0.8283...
Epoch: 8/250... Train Loss: 0.4900... Validation acc: 0.8306...
Epoch: 9/250... Train Loss: 0.4913... Validation acc: 0.8275...
Epoch: 10/250... Train Loss: 0.4891... Validation acc: 0.8280...
Epoch: 11/250... Train Loss: 0.4893... Validation acc: 0.8277...
Epoch: 12/250... Train Loss: 0.4883... Validation acc: 0.8307...
Epoch: 13/250... Train Loss: 0.4879... Validation acc: 0.8310...
Epoch: 14/250... Train Loss: 0.4873... Validation acc: 0.8298...
Epoch: 15/250... Train Loss: 0.4866... Valid

Epoch: 126/250... Train Loss: 0.4201... Validation acc: 0.8806...
Epoch: 127/250... Train Loss: 0.4199... Validation acc: 0.8812...
Epoch: 128/250... Train Loss: 0.4194... Validation acc: 0.8802...
Epoch: 129/250... Train Loss: 0.4199... Validation acc: 0.8833...
Epoch: 130/250... Train Loss: 0.4195... Validation acc: 0.8815...
Epoch: 131/250... Train Loss: 0.4196... Validation acc: 0.8830...
Epoch: 132/250... Train Loss: 0.4185... Validation acc: 0.8815...
Epoch: 133/250... Train Loss: 0.4184... Validation acc: 0.8840...
Epoch: 134/250... Train Loss: 0.4190... Validation acc: 0.8840...
Epoch: 135/250... Train Loss: 0.4186... Validation acc: 0.8837...
Epoch: 136/250... Train Loss: 0.4186... Validation acc: 0.8842...
Epoch: 137/250... Train Loss: 0.4185... Validation acc: 0.8837...
Epoch: 138/250... Train Loss: 0.4185... Validation acc: 0.8852...
Epoch: 139/250... Train Loss: 0.4177... Validation acc: 0.8830...
Epoch: 140/250... Train Loss: 0.4182... Validation acc: 0.8830...
Epoch: 141

[0.8572222222222222, 0.8855555555555555, 0.9119444444444446, 0.9316666666666668, 0.9744444444444444]
[0.8588888888888888, 0.8888888888888888, 0.9144444444444446, 0.9299999999999999, 0.9744444444444443]


In [14]:
net = cnn_lstm()
net.apply(init_weights)
res_best, res_final, best, final = onerun(net,fixlen_path)
print(res_best)
print(res_final)

creating synthetic data
smoothing data
making datasets
Epoch: 1/250... Train Loss: 0.6028... Validation acc: 0.7222...
Epoch: 2/250... Train Loss: 0.5915... Validation acc: 0.7222...
Epoch: 3/250... Train Loss: 0.5911... Validation acc: 0.7222...
Epoch: 4/250... Train Loss: 0.5905... Validation acc: 0.7222...
Epoch: 5/250... Train Loss: 0.5898... Validation acc: 0.7222...
Epoch: 6/250... Train Loss: 0.5910... Validation acc: 0.7222...
Epoch: 7/250... Train Loss: 0.5906... Validation acc: 0.7222...
Epoch: 8/250... Train Loss: 0.5903... Validation acc: 0.7222...
Epoch: 9/250... Train Loss: 0.5901... Validation acc: 0.7222...
Epoch: 10/250... Train Loss: 0.5888... Validation acc: 0.7222...
Epoch: 11/250... Train Loss: 0.5799... Validation acc: 0.7351...
Epoch: 12/250... Train Loss: 0.5501... Validation acc: 0.8077...
Epoch: 13/250... Train Loss: 0.5160... Validation acc: 0.8336...
Epoch: 14/250... Train Loss: 0.5029... Validation acc: 0.8365...
Epoch: 15/250... Train Loss: 0.5001... Valid

Epoch: 126/250... Train Loss: 0.4227... Validation acc: 0.8726...
Epoch: 127/250... Train Loss: 0.4223... Validation acc: 0.8770...
Epoch: 128/250... Train Loss: 0.4219... Validation acc: 0.8793...
Epoch: 129/250... Train Loss: 0.4208... Validation acc: 0.8804...
Epoch: 130/250... Train Loss: 0.4206... Validation acc: 0.8763...
Epoch: 131/250... Train Loss: 0.4196... Validation acc: 0.8804...
Epoch: 132/250... Train Loss: 0.4189... Validation acc: 0.8765...
Epoch: 133/250... Train Loss: 0.4178... Validation acc: 0.8811...
Epoch: 134/250... Train Loss: 0.4166... Validation acc: 0.8823...
Epoch: 135/250... Train Loss: 0.4159... Validation acc: 0.8895...
Epoch: 136/250... Train Loss: 0.4151... Validation acc: 0.8865...
Epoch: 137/250... Train Loss: 0.4134... Validation acc: 0.8926...
Epoch: 138/250... Train Loss: 0.4126... Validation acc: 0.8930...
Epoch: 139/250... Train Loss: 0.4117... Validation acc: 0.8973...
Epoch: 140/250... Train Loss: 0.4127... Validation acc: 0.8996...
Epoch: 141

[0.906388888888889, 0.9111111111111112, 0.9158333333333333, 0.9111111111111112, 0.9283333333333333]
[0.905277777777778, 0.91, 0.9144444444444443, 0.9094444444444444, 0.9272222222222221]
